# prep work for a little demo

Scenario: find a set of diaObjects that all are peaking around the same time.

## set up

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.cosmology import FlatLambdaCDM
from lsst.rsp import get_tap_service

# from astropy.coordinates import SkyCoord
# from astropy.coordinates import match_coordinates_sky
# import astropy.units as u

In [ ]:
service = get_tap_service("tap")

In [ ]:
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

In [ ]:
plt.style.use('tableau-colorblind10')

plot_filter_labels = ['u', 'g', 'r', 'i', 'z', 'y']
plot_filter_colors = {'u': '#56b4e9', 'g': '#008060', 'r': '#ff4000',
                      'i': '#850000', 'z': '#6600cc', 'y': '#000000'}
plot_filter_symbols = {'u': 'o', 'g': '^', 'r': 'v', 'i': 's', 'z': '*', 'y': 'p'}

## identify potential SNIa to use

Use the same method for identifying potential SNIa as in NB 07a.

In [ ]:
redshift_min = 0.1
redshift_max = 0.3

snia_peak_mag = -19.0
snia_peak_mag_range = 0.5

snia_peak_mr_min = cosmo.distmod(redshift_min).value + snia_peak_mag - snia_peak_mag_range
snia_peak_mr_max = cosmo.distmod(redshift_max).value + snia_peak_mag + snia_peak_mag_range

snia_peak_mg_max = 24.0
snia_peak_mi_max = 24.0

snia_ampl_mr_min = 1.5
snia_ampl_mr_max = 5.5

snia_nDiaSources_min = 15
snia_nDiaSources_max = 100

snia_duration_min = 50
snia_duration_max = 300

Search for `diaObjects` that have suitable light curve summary parameters to potentially be SNIa.

In [ ]:
DiaObjs = service.search("SELECT TOP 10000 "
                         "ra, decl, diaObjectId, nDiaSources, "
                         "scisql_nanojanskyToAbMag(rPSFluxMin) AS rMagMax, "
                         "scisql_nanojanskyToAbMag(rPSFluxMax) AS rMagMin, "
                         "scisql_nanojanskyToAbMag(gPSFluxMax) AS gMagMin, "
                         "scisql_nanojanskyToAbMag(iPSFluxMax) AS iMagMin "
                         "FROM dp02_dc2_catalogs.DiaObject "
                         "WHERE nDiaSources > "+str(snia_nDiaSources_min)+" "
                         "AND nDiaSources < "+str(snia_nDiaSources_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMax) > "+str(snia_peak_mr_min)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMax) < "+str(snia_peak_mr_max)+" "
                         "AND scisql_nanojanskyToAbMag(gPSFluxMax) < "+str(snia_peak_mg_max)+" "
                         "AND scisql_nanojanskyToAbMag(iPSFluxMax) < "+str(snia_peak_mi_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) < "+str(snia_ampl_mr_max)+" "
                         "AND scisql_nanojanskyToAbMag(rPSFluxMin)"
                         " - scisql_nanojanskyToAbMag(rPSFluxMax) > "+str(snia_ampl_mr_min)+" ").to_table()

In [ ]:
DiaObjs['rMagAmp'] = DiaObjs['rMagMax'] - DiaObjs['rMagMin']

In [ ]:
temp = np.unique(np.asarray(DiaObjs['diaObjectId'][0:-1]))
print(len(temp))
tempstring = "(" + ','.join(['%22i' % num for num in temp]) + ")"
# tempstring

Get the light curves from `DiaSource` so that duration and date of r-band maximum can be determined.

In [ ]:
DiaSrcs = service.search("SELECT diao.diaObjectId, dias.midPointTai, dias.filterName, "
                         "scisql_nanojanskyToAbMag(dias.psFlux) AS psAbMag "
                         "FROM dp02_dc2_catalogs.DiaObject AS diao "
                         "JOIN dp02_dc2_catalogs.DiaSource AS dias "
                         "ON diao.diaObjectId = dias.diaObjectId "
                         "WHERE diao.diaObjectId IN "+tempstring).to_table()

In [ ]:
DiaObjs['duration'] = np.zeros(len(DiaObjs), dtype='float')
DiaObjs['mjd_rMagMin'] = np.zeros(len(DiaObjs), dtype='float')
DiaObjs['nDiaSrcs'] = np.zeros(len(DiaObjs), dtype='int')
DiaObjs['nrDiaSrcs'] = np.zeros(len(DiaObjs), dtype='int')
DiaObjs['nppDiaSrcs'] = np.zeros(len(DiaObjs), dtype='float')

for i, tid in enumerate(temp):
    # index the diaObject for this element in temp
    ix = np.where(DiaObjs['diaObjectId'] == tid)[0]
    # index the diaSources for this diaObject
    tx = np.where(DiaSrcs['diaObjectId'] == tid)[0]
    DiaObjs['nDiaSrcs'][ix] = len(tx)
    DiaObjs['duration'][ix] = np.max(DiaSrcs['midPointTai'][tx]) - np.min(DiaSrcs['midPointTai'][tx])
    # index the r-band diaSources for this diaObject
    trx = np.where((DiaSrcs['diaObjectId'] == tid) & (DiaSrcs['filterName'] == 'r'))[0]
    DiaObjs['nrDiaSrcs'][ix] = len(trx)
    mx = np.argmin(DiaSrcs['psAbMag'][trx])
    DiaObjs['mjd_rMagMin'][ix] = DiaSrcs['midPointTai'][trx[mx]]
    # index the pre-"peak" diaSources for this diaObject
    tpx = np.where(DiaSrcs['midPointTai'][tx] <= DiaObjs['mjd_rMagMin'][i])[0]
    DiaObjs['nppDiaSrcs'][ix] = len(tpx)
    del ix, tx, trx, mx, tpx

In [ ]:
DiaObjs

In [ ]:
tx = np.where((DiaObjs['duration'] > snia_duration_min)
              & (DiaObjs['duration'] < snia_duration_max))[0]
print(len(tx))

OK so in total there are 331 SNIa that pass these crude cuts for low-z SNIa.

Which dates might have many SNIa peaking?

In [ ]:
fig = plt.figure(figsize=(3, 2))
plt.hist(DiaObjs['mjd_rMagMin'][tx], bins=100)
plt.show()

In [ ]:
fig = plt.figure(figsize=(3, 2))
ttx = np.where((DiaObjs['mjd_rMagMin'][tx] > 60800) & (DiaObjs['mjd_rMagMin'][tx] < 61100))[0]
plt.hist(DiaObjs['mjd_rMagMin'][tx[ttx]], bins=100)
plt.show()
del ttx

In [ ]:
fig = plt.figure(figsize=(3, 2))
ttx = np.where((DiaObjs['mjd_rMagMin'][tx] > 60950) & (DiaObjs['mjd_rMagMin'][tx] < 60975))[0]
plt.hist(DiaObjs['mjd_rMagMin'][tx[ttx]]-60950, bins=25)
plt.xlabel('MJD - 60950')
plt.show()
del ttx

In [ ]:
ttx = np.where((DiaObjs['mjd_rMagMin'][tx] > 60960) & (DiaObjs['mjd_rMagMin'][tx] < 60962))[0]
print(len(ttx))

OK how many of these have a lot of pre-peak light curve detections.

In [ ]:
fig = plt.figure(figsize=(3, 2))
plt.hist(DiaObjs['nppDiaSrcs'][tx[ttx]])
plt.show()

Distribution of durations.

In [ ]:
fig = plt.figure(figsize=(3, 2))
plt.hist(DiaObjs['duration'][tx[ttx]])
plt.show()

And what are their brightest r-band detections.

In [ ]:
fig = plt.figure(figsize=(3, 2))
plt.hist(DiaObjs['rMagMin'][tx[ttx]])
plt.show()

In [ ]:
fig = plt.figure(figsize=(3, 2))
plt.plot(DiaObjs['rMagMin'][tx[ttx]], DiaObjs['nppDiaSrcs'][tx[ttx]], 'o', alpha=0.4, mew=0)
plt.show()

In [ ]:
tttx = np.where((DiaObjs['nppDiaSrcs'][tx[ttx]] >= 5)
               & (DiaObjs['rMagMin'][tx[ttx]] >= 20)
               & (DiaObjs['rMagMin'][tx[ttx]] <= 22))[0]
print(len(tttx))

Store the `diaObjectId` of our 31 objects of interest.

In [ ]:
my_list = np.asarray(DiaObjs['diaObjectId'][tx[ttx[tttx]]], dtype='int')
my_list

In [ ]:
my_list_string = "(" + ','.join(['%22i' % num for num in my_list]) + ")"
my_list_string

In [ ]:
del tx, ttx, tttx

## which potential SNIa are the best to use for the demo?

Want objects that have demonstrated a rise, and have a bunch of pre-peak light curve detections.

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 6), sharey=False, sharex=False)

x = 0

for i in range(2):
    for j in range(2):
        if x < len(my_list):
            ix = np.where(DiaObjs['diaObjectId'] == my_list[x])[0]
            tx = np.where(DiaSrcs['diaObjectId'] == my_list[x])[0]
            print('%2i %2i %2i %20i %4.1f %4i ' 
                  % (x, i, j, my_list[x], DiaObjs['duration'][ix], len(tx)))
            ax[i, j].axvline(DiaObjs['mjd_rMagMin'][ix] - 60960, color=plot_filter_colors['r'])
            for f, filt in enumerate(plot_filter_labels):
                fx = np.where(DiaSrcs['filterName'][tx] == filt)[0]
                ax[i, j].plot(DiaSrcs['midPointTai'][tx[fx]] - 60960, 
                              DiaSrcs['psAbMag'][tx[fx]],
                              plot_filter_symbols[filt], ms=10, mew=0, alpha=0.5,
                              color=plot_filter_colors[filt])
                del fx
            del ix, tx
            
            ax[i, j].invert_yaxis()
            ax[i, j].set_title(my_list[x])
            if i == 1:
                ax[i, j].xaxis.set_label_text('MJD (days)')
            if j == 0:
                ax[i, j].yaxis.set_label_text('mag')
        x += 1

plt.tight_layout()
plt.show()

OK so these will be the four that the brokers "find" and that we need to retrieve host info for.

## what is their host situation

first make sure there are at least some nearby things to check out

In [ ]:
for i, tid in enumerate(my_list):
    ix = np.where(DiaObjs['diaObjectId'] == tid)[0]
    print(i, tid)
    sra = str(float(DiaObjs['ra'][ix]))
    sdec = str(float(DiaObjs['decl'][ix]))
    srad = str(np.round(10.0 / 3600.0, 5))
    query = "SELECT objectId, coord_ra, coord_dec, x, y, refExtendedness, "\
    "shape_xx, shape_xy, shape_yy, r_kronRad, i_kronRad "\
    "FROM dp02_dc2_catalogs.Object "\
    "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), "\
    "CIRCLE('ICRS', "+sra+", "+sdec+", "+srad+")) = 1 "\
    "AND detect_isPrimary = 1 "
    print(sra, sdec, srad)
    Objs = service.search(query).to_table()
    print('number of Objects within 20" = ', len(Objs))